In [ ]:
from googleapiclient.discovery import build
import csv
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import scipy
import networkx as nx
from itertools import combinations
import math
import numpy as np
from collections import defaultdict
from statsmodels.distributions.empirical_distribution import ECDF
import json
from collections import defaultdict

In [ ]:
# Canali target da cui raccogliere i video
channels = [
    "Fanpage.it",
    "La Repubblica",
    "FABRIZIO CORONA",
    "Gianluca Spina",
    "Gianmarco Zagato",
    "La7 Attualità",
    "Rai",
    "Bugalalla Crime",
    "DarkSide - Storia Segreta d'Italia"
]

# Intervallo temporale per i video (due settimana prima dalla riapertura del caso fino ai giorni odierni)
begin_date = datetime(2025, 2, 25)
end_date = datetime(2025, 6, 8)

In [ ]:
# Crea struttura Comment
class Comment:
    def __init__(self, id, video_id, content, author, date, likes, reply_to_id=None):
        self.id = id
        self.video_id = video_id
        self.content = content
        self.author = author
        self.date = datetime.strptime(date, "%Y-%m-%d")
        self.likes = int(likes)
        self.reply_to_id = reply_to_id

    def __repr__(self):
        return f"<Comment by {self.author} on {self.date.strftime('%Y-%m-%d')}>"


In [ ]:
# Carica file JSON unificati
with open("./garlasco_comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)

with open("./garlasco_videos.json", "r", encoding="utf-8") as f:
    videos = json.load(f)

In [ ]:
# Mappa video_id → canale
Channel_of = {v["video_id"]: v["channel"] for v in videos}

# Mappa canale → lista video_id
Videos_of = defaultdict(list)
for v in videos:
    Videos_of[v["channel"]].append(v["video_id"])

# Mappa video_id → data
Date_of_video = {v["video_id"]: v["published_at"] for v in videos}

# Inizializza dizionari ausiliari
Commenters_of_video = defaultdict(set)
Videos_commented_by = defaultdict(set)
Channels_commented_by = defaultdict(set)
Commenters_of_channel = defaultdict(set)
Comments_of_channel = defaultdict(list)
Comments_by_user = defaultdict(list)

# Popola strutture dai commenti
for c in comments:
    video_id = c["video_id"]
    author = c["author"]
    channel = Channel_of.get(video_id, "Unknown")

    comment_obj = Comment(
        id=c["comment_id"],
        video_id=video_id,
        content=c["content"],
        author=author,
        date=c["date"],
        likes=c["likes"],
        reply_to_id=c.get("reply_to_id")
    )

    Commenters_of_video[video_id].add(author)
    Videos_commented_by[author].add(video_id)
    Channels_commented_by[author].add(channel)
    Commenters_of_channel[channel].add(author)
    Comments_of_channel[channel].append(comment_obj)
    Comments_by_user[author].append(comment_obj)

# Debug: stampa sommaria
print(f"Video totali: {len(Channel_of)}")
print(f"Utenti unici: {len(Comments_by_user)}")
print(f"Canali totali: {len(Videos_of)}")
